In [265]:
import requests
from datetime import date
import datetime
import PIL
from PIL import Image
import PIL.ImageOps
import os
import numpy
import pandas as pd
from requests.exceptions import ConnectionError
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

In [2]:
# Save Image
def save_image(image, path):
    if os.path.exists(path):
        os.remove(path)
    
    image.save(path, 'JPEG')

def getdate(i):
    p_date = date.today()
    days = datetime.timedelta(i+1)
    e_date = p_date - days
    n_date = e_date.strftime("%Y-%m-%d")
    return n_date

def openImage(i):
    extension = '.JPEG'
    n_date = getdate(i)
    file_name = n_date + extension
    image = PIL.Image.open(file_name)
    return image

In [5]:
try:
    for i in range(10):
        URL_1 = 'https://wvs.earthdata.nasa.gov/api/v1/snapshot?REQUEST=GetSnapshot&LAYERS=VIIRS_SNPP_CorrectedReflectance_TrueColor&CRS=EPSG:4326&TIME='
        URL_2 = '&WRAP=DAY&BBOX=5.505452,79.264304,10.049224,82.491415&FORMAT=image/jpeg&WIDTH=1469&HEIGHT=2068&AUTOSCALE=TRUE&ts=1604169569770'
        n_date = getdate(i)
        URL = URL_1+n_date+URL_2
        r = requests.get(URL,allow_redirects=True)
        f_name = n_date+'.JPEG'
        open(f_name,'wb').write(r.content)
        print('saved ' + f_name)
except ConnectionError:
    print("Check your Internet Connection and try again")

saved 2020-10-31.JPEG
saved 2020-10-30.JPEG
saved 2020-10-29.JPEG
saved 2020-10-28.JPEG
saved 2020-10-27.JPEG
saved 2020-10-26.JPEG
saved 2020-10-25.JPEG
saved 2020-10-24.JPEG
saved 2020-10-23.JPEG
saved 2020-10-22.JPEG


In [353]:
train_images = []
for i in range(10):
    n_date = getdate(i)
    image = openImage(i)
    greyscale_img = image.convert(mode="1", dither=Image.NONE)
    greyscale_img = greyscale_img.convert('L')
    inverted_image = PIL.ImageOps.invert(greyscale_img)
    n_name = n_date+'.JPEG'
    save_image(inverted_image,n_name)
    print('saved '+n_name)
    image_array = cv2.imread('2020-10-22.JPEG')
    train_images.append(image_array)

saved 2020-10-31.JPEG
saved 2020-10-30.JPEG
saved 2020-10-29.JPEG
saved 2020-10-28.JPEG
saved 2020-10-27.JPEG
saved 2020-10-26.JPEG
saved 2020-10-25.JPEG
saved 2020-10-24.JPEG
saved 2020-10-23.JPEG
saved 2020-10-22.JPEG


In [355]:
train_images = numpy.array(train_images)

In [356]:
train_images.shape

(10, 2068, 1455, 3)

In [303]:
def count_blk_cover(i):
    n_date = getdate(i)
    im = openImage(i)
    print('Processing Image '+str(im.filename))
    count_blacks = 0
    count_whites = 0
    width, height = im.size
    t_px=width*height

    for h in range(height):
        string = ''
        for w in range(width):
            cordinates = w,h
            val = im.getpixel(cordinates)
            if(val>0):
                count_blacks += 1
    return count_blacks

def categorize_clouds(count_blacks):
    cloud_category = 'None'
    if (count_blacks/t_px)<0.1:
        cloud_category = '1'
    elif (count_blacks/t_px)<0.5:
        cloud_category = '2'
    elif (count_blacks/t_px)<0.9:
        cloud_category = '3'
    else:
        cloud_category = '4'
    return cloud_category


In [375]:
cloud_cats = numpy.array([],dtype='U')
for j in range(10):
    black_count = count_blk_cover(j)
    category = categorize_clouds(black_count)
    cloud_cats = numpy.insert(cloud_cats,len(cloud_cats),category)

processing image 2020-10-31.JPEG
processing image 2020-10-30.JPEG
processing image 2020-10-29.JPEG
processing image 2020-10-28.JPEG
processing image 2020-10-27.JPEG
processing image 2020-10-26.JPEG
processing image 2020-10-25.JPEG
processing image 2020-10-24.JPEG
processing image 2020-10-23.JPEG
processing image 2020-10-22.JPEG


In [376]:
cloud_cats

array(['3', '2', '2', '2', '3', '2', '3', '2', '2', '2'], dtype='<U1')

In [380]:
train_images = train_images.reshape(-1, (2068*1455))

In [381]:
len(cloud_cats)

10

In [382]:
train_images.shape

(30, 3008940)

In [372]:
#building the model
# 3 layers, 2 layers with 64 neurons and relu function
# 1 layer with 10 neurons and softmax function
model = Sequential()
model.add(Dense(64, activation = 'relu',input_dim=9026820))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1 , activation = 'softmax'))

In [373]:
#Compile the model
#The loss function measures how well the model did on training, and the tries to improve on it using the optimizer
model.compile(
    optimizer='adam',
    loss = 'categorical_crossentropy', #Classes that are greater than 2
    metrics = ['accuracy']
)

In [379]:
model.fit(
    train_images,
        to_categorical(cloud_cats), #Ex. 2 it expects [0,0,1,0,0,0,0,0,0]
        epochs = 5, #number of iterations over the entire dataset to train on
        batch_size = 10 #number of samples per gradient update for training    
)

Epoch 1/5


ResourceExhaustedError: OOM when allocating tensor with shape[9026820,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu
	 [[{{node training_2/Adam/mul_2}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [339]:
# del(train_images)